In [ ]:
!pip install librosa
!dir
! pip install -q tensorflow-text
! pip install tensorflow_addons
!pip install tensorflow-hub tensorflow
!pip install lazypredict



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
sample_data
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 65.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.3/588.3 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 38.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 40.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.2/439.2 KB 24.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 47.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
#-------------------------------------------------
import IPython.display as ipd
import librosa
import librosa.display
import os
#-------------------------------------------------
import pandas as pd
#-------------------------------------------------
import numpy as np
from tqdm import tqdm
#-------------------------------------------------
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import models, layers
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import tensorflow_addons as tfa
import tensorflow as keras
from sklearn import metrics
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
#-------------------------------------------------
from sklearn.model_selection import train_test_split
#-------------------------------------------------
from tensorflow.keras.callbacks import ModelCheckpoint
from datetime import datetime 
#-------------------------------------------------
from xgboost import XGBClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.svm import NuSVC
from sklearn.ensemble import ExtraTreesClassifier
import lightgbm as lgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import Perceptron
from sklearn.neighbors import NearestCentroid
from sklearn.ensemble import BaggingClassifier
from sklearn import svm
from lazypredict.Supervised import LazyClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier
#-------------------------------------------------
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import f1_score

In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [ ]:
#Choose Question
question=1

if(question==1):
  number_of_audios=1672
elif(question==2):
  number_of_audios=1278
elif(question==3):
  number_of_audios=1808
elif(question==4):
  number_of_audios=1657
elif(question==5):
  number_of_audios=1795
elif(question==6):
  number_of_audios=1797
elif(question==7):
  number_of_audios=1799
elif(question==8):
  number_of_audios=1799
elif(question==9):
  number_of_audios=1798
elif(question==10):
  number_of_audios=1640

In [ ]:
audio = ["" for x in range(number_of_audios)]
for i in range(1,number_of_audios+1):
    path ='/content/drive/MyDrive/Speech Dataset/Question'+str(question)+'/wav/'+str(question)+'-'+str(i)+'.wav'
    #path=path[:0] + path[1:]
    audio[i-1]=path

In [ ]:
ref_df=pd.read_csv('/content/drive/MyDrive/Speech Dataset/sas-ar.csv')
ref_df=ref_df[ref_df['EssaySet']==question]
ref_df.reset_index(inplace=True)

In [ ]:
df=pd.DataFrame(audio, columns=['Audios'])
df['class']=ref_df['Score1']

In [ ]:
def features_extractor_rms_2d(file):
    audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
    features = librosa.feature.rms(y=audio)
    features = librosa.power_to_db(features , ref=np.max)
    scaled_features = np.mean(features.T,axis=0)
    return scaled_features
    

def features_extractor_mel_spect_3d(file):
    audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
    features = librosa.feature.melspectrogram(y=audio, sr=sample_rate, n_fft=2048, hop_length=1024)
    features = librosa.power_to_db(features , ref=np.max)

    features=features.tolist()
    for j in range(len(features)):
      features[j]
      for i in range(1300-len(features[j])):
        features[j].append(0)
    features=np.array(features)
    return features
def features_extractor_mel_spect_2d(file):
    audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
    features = librosa.feature.melspectrogram(y=audio, sr=sample_rate, n_fft=2048, hop_length=1024)
    features = librosa.power_to_db(features , ref=np.max)
    scaled_features = np.mean(features.T,axis=0)
    return scaled_features
    
def features_extractor_mfcc_3d(file):
    audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
    features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
  
    features=features.tolist()
    for j in range(len(features)):
      features[j]
      for i in range(2600-len(features[j])):
        features[j].append(0)
    features=np.array(features)
    return features
def features_extractor_mfcc_2d(file):
    audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
    features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
    scaled_features = np.mean(features.T,axis=0)
    
    return scaled_features

In [ ]:
### Now we iterate through every audio file and extract features 
extracted_features=[]
for index_num,row in tqdm(df.iterrows()):
    file_name = row['Audios']
    final_class_labels=row["class"]
    #data=features_extractor_mfcc_2d(file_name)
    data=features_extractor_mel_spect_2d(file_name)
    #data=features_extractor_rms_2d(file_name)
    extracted_features.append([data,final_class_labels])

extracted_features_df=pd.DataFrame(extracted_features,columns=['feature','class'])

1672it [07:00,  3.98it/s]


In [ ]:
X=np.array(extracted_features_df['feature'].tolist())
y=np.array(extracted_features_df['class'].tolist())

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)
num_labels=len(np.unique(y_train))
print(np.unique(y_train, return_counts=True))

(array([0, 1, 2, 3]), array([304, 343, 425, 265]))


# vGGish

In [ ]:
# Load the VGGish model from TensorFlow Hub
model = hub.load("https://tfhub.dev/google/vggish/1")

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Dense, Dropout, Flatten, GRU, SimpleRNN, LSTM, Bidirectional, Activation, TimeDistributed
import keras,os
from keras.models import Sequential


# Preprocess your audio data and labels
# audio_data should be a numpy array with shape (num_samples, num_features)
# labels should be a numpy array with shape (num_samples,)
  #audio_data, labels = all_mfccs,labels

# Convert the audio data and labels to tensors
#audio_data_tensor = tf.convert_to_tensor(audio_data)
#labels_tensor = tf.convert_to_tensor(labels)

    # audio_data_tensor = audio_data
    # labels_tensor = labels
    # labels_tensor=labels_tensor[:10]

# Fine-tune the model by adding a classification layer on top of it
model = keras.Sequential([
    model,
    keras.layers.Dense(200,input_shape=(40,2600,1)),
    keras.layers.Dense(64, activation="softmax"),
    keras.layers.Dense(4, activation="softmax")
])

 #Compile the model
# "sparse_categorical_crossentropy"
#model.compile(optimizer="adam", loss='sparse_categorical_crossentropy', metrics=["accuracy"])
# Train the model
#model.fit(audio_data_tensor, labels_tensor, epochs=10)

# Evaluate the model on the test data
#test_loss, test_acc = model.evaluate(test_audio_data_tensor, test_labels_tensor)
#print(f"Test accuracy: {test_acc}")

TypeError: ignored

In [ ]:
audio_data_tensor=np.array(audio_data_tensor)
labels_tensor=np.array(labels_tensor)
#labelencoder=LabelEncoder()
#labels_tensor=to_categorical(labelencoder.fit_transform(labels_tensor))

In [ ]:
audio_data_tensor=audio_data_tensor.reshape(audio_data_tensor.shape[0],audio_data_tensor.shape[1],audio_data_tensor.shape[2],1)

# DenseNet

In [ ]:
import tensorflow 

import pandas as pd
import numpy as np
import os
import keras
import random
import cv2
import math
import seaborn as sns

from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt

from tensorflow.keras.layers import Dense,GlobalAveragePooling2D,Convolution2D,BatchNormalization
from tensorflow.keras.layers import Flatten,MaxPooling2D,Dropout

from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.applications.densenet import preprocess_input

from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,img_to_array

from tensorflow.keras.models import Model

from tensorflow.keras.optimizers import Adam

from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

import warnings
warnings.filterwarnings("ignore")

In [ ]:
X=np.array(extracted_features_df['feature'].tolist())
y=np.array(extracted_features_df['class'].tolist())
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)

X_train=X_train.reshape(X_train.shape[0],X_train.shape[1],X_train.shape[2],1)
X_test=X_test.reshape(X_test.shape[0],X_test.shape[1],X_test.shape[2],1)

In [ ]:
model_d=DenseNet121(weights=None,include_top=False, input_shape=(40,2600,1))

x=model_d.output

x= GlobalAveragePooling2D()(x)
x= BatchNormalization()(x)
x= Dropout(0.5)(x)
x= Dense(128,activation='relu')(x) 
x= Dense(64,activation='relu')(x) 
x= BatchNormalization()(x)
x= Dropout(0.5)(x)

preds=Dense(4,activation='softmax')(x) #FC-layer

In [ ]:
model=Model(inputs=model_d.input,outputs=preds)

In [ ]:
for layer in model.layers[:-8]:
    layer.trainable=False
    
for layer in model.layers[-8:]:
    layer.trainable=True

In [ ]:

model.compile(optimizer='Adam',loss='categorical_crossentropy',
              metrics=[tf.keras.metrics.Recall(),tf.keras.metrics.Precision(),tfa.metrics.CohenKappa(num_classes=num_labels, sparse_labels=True, weightage="quadratic"),"accuracy"]
              )

In [ ]:
y_train = np.array(y_train)
mlb = LabelBinarizer()
y_train = mlb.fit_transform(y_train)

In [ ]:
anne = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=5, verbose=1, min_lr=1e-3)
checkpoint = ModelCheckpoint('model.h5', verbose=1, save_best_only=True)

datagen = ImageDataGenerator(zoom_range = 0.2, horizontal_flip=True, shear_range=0.2)


datagen.fit(X_train)
# Fits-the-model
history = model.fit_generator(datagen.flow(X_train, y_train, batch_size=8),
               steps_per_epoch=X_train.shape[0] //128,
               epochs=50,
               verbose=2,
               callbacks=[anne, checkpoint],
               validation_data=(X_train, y_train))

In [ ]:
op = keras.optimizers.Adam(learning_rate=0.001)
loss = keras.losses.SparseCategoricalCrossentropy()
model.compile(optimizer=op,
              loss=loss,
              metrics=[tfa.metrics.CohenKappa(weightage="quadratic", sparse_labels=True, num_classes=num_labels),"accuracy"])

In [ ]:
model.fit(X_train, y_train, epochs=10, validation_split=0.2, batch_size=8, shuffle=True)

Epoch 1/10


KeyboardInterrupt: ignored

In [ ]:
print(model.evaluate(X_test,y_test))

5/5 [==============================] - 52s 10s/step - loss: 1.4601 - cohen_kappa: 0.1788 - accuracy: 0.3000
[1.4600660800933838, 0.17882800102233887, 0.30000001192092896]


# LSTM


In [ ]:
X=np.array(extracted_features_df['feature'].tolist())
y=np.array(extracted_features_df['class'].tolist())
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)

X_train=np.expand_dims(X_train,axis=1)
X_test=np.expand_dims(X_test,axis=1)

In [ ]:
X_train.shape

(1446, 1, 128)

In [ ]:
tf.keras.backend.clear_session()
epochs = 10
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=50)
lstm_model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(X_train.shape[1],X_train.shape[2])),
        tf.keras.layers.LSTM(128,activation="relu", return_sequences=True), #,recurrent_dropout=0.2
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.LSTM(64 ,activation="relu"), #, recurrent_dropout=0.2
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(num_labels, activation="softmax")
])
op = keras.optimizers.Adam(learning_rate=0.001)
loss = keras.losses.SparseCategoricalCrossentropy()
lstm_model.compile(optimizer=op,
              loss=loss,
              metrics=[tfa.metrics.CohenKappa(weightage="quadratic", sparse_labels=True, num_classes=num_labels),"accuracy"])
lstm_history = lstm_model.fit(X_train, y_train,epochs=epochs, validation_split=0.1, batch_size=64, callbacks=[callback], shuffle=True)


Epoch 1/10
21/21 [==============================] - 8s 44ms/step - loss: 2.2842 - cohen_kappa: -0.0124 - accuracy: 0.4420 - val_loss: 1.0155 - val_cohen_kappa: 0.0000e+00 - val_accuracy: 0.5586
Epoch 2/10
21/21 [==============================] - 0s 10ms/step - loss: 1.0912 - cohen_kappa: 0.0419 - accuracy: 0.4996 - val_loss: 0.9937 - val_cohen_kappa: 0.0000e+00 - val_accuracy: 0.5586
Epoch 3/10
21/21 [==============================] - 0s 12ms/step - loss: 1.0470 - cohen_kappa: -0.0039 - accuracy: 0.5327 - val_loss: 0.9904 - val_cohen_kappa: 0.0000e+00 - val_accuracy: 0.5586
Epoch 4/10
21/21 [==============================] - 0s 11ms/step - loss: 1.0024 - cohen_kappa: 0.0082 - accuracy: 0.5596 - val_loss: 0.9948 - val_cohen_kappa: 0.0000e+00 - val_accuracy: 0.5586
Epoch 5/10
21/21 [==============================] - 0s 10ms/step - loss: 0.9973 - cohen_kappa: 0.0032 - accuracy: 0.5596 - val_loss: 1.0021 - val_cohen_kappa: 0.0000e+00 - val_accuracy: 0.5586
Epoch 6/10
21/21 [===============

In [ ]:
print(lstm_model.evaluate(X_test,y_test))

12/12 [==============================] - 0s 4ms/step - loss: 1.0039 - cohen_kappa: 0.0000e+00 - accuracy: 0.5442
[1.003897786140442, 0.0, 0.5441988706588745]


# RNN

In [ ]:
tf.keras.backend.clear_session()
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=100)
epoch = 10
rnn_model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(1,X_train.shape[2])),
        tf.keras.layers.SimpleRNN(128,activation="sigmoid", return_sequences=True),
        tf.keras.layers.Dropout(.2),
        tf.keras.layers.SimpleRNN(64 ,activation="sigmoid"),
        #tf.keras.layers.Dropout(.2),
        tf.keras.layers.Dense(num_labels, activation="softmax"),
])
op = keras.optimizers.Adam(learning_rate=0.001)
loss = keras.losses.SparseCategoricalCrossentropy()
rnn_model.compile(optimizer=op,
              loss=loss,
              metrics=[tfa.metrics.CohenKappa(num_classes=num_labels, sparse_labels=True, weightage="quadratic"),"accuracy"])
rnn_history = rnn_model.fit(X_train, y_train,epochs=epoch, validation_split=0.1, batch_size=8, callbacks=[callback])

Epoch 1/10
148/148 [==============================] - 4s 8ms/step - loss: 1.0450 - cohen_kappa: -0.0183 - accuracy: 0.4644 - val_loss: 1.0114 - val_cohen_kappa: 0.0000e+00 - val_accuracy: 0.5227
Epoch 2/10
148/148 [==============================] - 1s 4ms/step - loss: 1.0487 - cohen_kappa: 0.0082 - accuracy: 0.4551 - val_loss: 1.0238 - val_cohen_kappa: 0.0000e+00 - val_accuracy: 0.5227
Epoch 3/10
148/148 [==============================] - 1s 4ms/step - loss: 1.0363 - cohen_kappa: -0.0035 - accuracy: 0.4415 - val_loss: 1.0141 - val_cohen_kappa: 0.0000e+00 - val_accuracy: 0.5227
Epoch 4/10
148/148 [==============================] - 1s 3ms/step - loss: 1.0365 - cohen_kappa: -0.0194 - accuracy: 0.4610 - val_loss: 1.0124 - val_cohen_kappa: 0.0000e+00 - val_accuracy: 0.5227
Epoch 5/10
148/148 [==============================] - 0s 3ms/step - loss: 1.0374 - cohen_kappa: 0.0097 - accuracy: 0.4669 - val_loss: 1.0234 - val_cohen_kappa: 0.0000e+00 - val_accuracy: 0.5227
Epoch 6/10
148/148 [=======

In [ ]:
print(rnn_model.evaluate(X_test,y_test))

11/11 [==============================] - 0s 4ms/step - loss: 1.0533 - cohen_kappa: 0.0000e+00 - accuracy: 0.4268
[1.0532646179199219, 0.0, 0.4268292784690857]


# Bidirectional

In [ ]:
epochs = 10
tf.keras.backend.clear_session()
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=50)
bi_model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(1,X_train.shape[2])),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128,activation="sigmoid", return_sequences=True)),
        tf.keras.layers.Dropout(.2),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64 ,activation="sigmoid")),
        #tf.keras.layers.Dropout(.2),
        tf.keras.layers.Dense(num_labels, activation="softmax"),
])
op = keras.optimizers.Adam(learning_rate=0.001)
loss = keras.losses.SparseCategoricalCrossentropy()
bi_model.compile(optimizer=op,
              loss=loss,
              metrics=[tfa.metrics.CohenKappa(weightage="quadratic", sparse_labels=True, num_classes=num_labels),"accuracy"])
bi_history = bi_model.fit(X_train, y_train,epochs=epochs, validation_split=0.1, batch_size=64, shuffle=True, callbacks=[callback])

Epoch 1/10
19/19 [==============================] - 7s 70ms/step - loss: 1.0498 - cohen_kappa: -6.5243e-04 - accuracy: 0.4407 - val_loss: 1.0512 - val_cohen_kappa: 0.0000e+00 - val_accuracy: 0.2955
Epoch 2/10
19/19 [==============================] - 0s 19ms/step - loss: 1.0356 - cohen_kappa: 0.0080 - accuracy: 0.4508 - val_loss: 1.0149 - val_cohen_kappa: 0.0000e+00 - val_accuracy: 0.5227
Epoch 3/10
19/19 [==============================] - 0s 18ms/step - loss: 1.0334 - cohen_kappa: -0.0041 - accuracy: 0.4695 - val_loss: 1.0170 - val_cohen_kappa: 0.0000e+00 - val_accuracy: 0.5227
Epoch 4/10
19/19 [==============================] - 0s 18ms/step - loss: 1.0357 - cohen_kappa: 0.0000e+00 - accuracy: 0.4754 - val_loss: 1.0250 - val_cohen_kappa: 0.0000e+00 - val_accuracy: 0.5227
Epoch 5/10
19/19 [==============================] - 0s 18ms/step - loss: 1.0262 - cohen_kappa: -0.0038 - accuracy: 0.4754 - val_loss: 1.0155 - val_cohen_kappa: 0.0000e+00 - val_accuracy: 0.5227
Epoch 6/10
19/19 [======

In [ ]:
print(bi_model.evaluate(X_test,y_test))

11/11 [==============================] - 0s 5ms/step - loss: 1.0522 - cohen_kappa: 0.0000e+00 - accuracy: 0.4268
[1.0522081851959229, 0.0, 0.4268292784690857]


# Deep learning


In [ ]:
X=np.array(extracted_features_df['feature'].tolist())
y=np.array(extracted_features_df['class'].tolist())
labelencoder=LabelEncoder()
y=to_categorical(labelencoder.fit_transform(y))
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)
X_train=np.expand_dims(X_train,axis=1)
X_test=np.expand_dims(X_test,axis=1)

In [ ]:
model=Sequential()
###first layer
model.add(Dense(200,input_shape=(X_train.shape[1],X_train.shape[2])))
model.add(Activation('relu'))
model.add(Dropout(0.2))
###second layer
model.add(Dense(200))
model.add(Activation('relu'))
model.add(Dropout(0.2))
###third layer
# model.add(Dense(200))
# model.add(Activation('relu'))
# model.add(Dropout(0.5))

###final layer
model.add(Dense(num_labels))
model.add(Activation('softmax'))


model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])


In [ ]:
#checkpointer = ModelCheckpoint(filepath='saved_models/audio_classification.hdf5', verbose=1, save_best_only=True)
#checkpointer = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=50)
start = datetime.now()
model.fit(X_train, y_train, batch_size=4, epochs=10, validation_split=0.2, verbose=1)
#model.fit(X_train, y_train,epochs=epochs, validation_split=0.2, batch_size=2, callbacks=[callback], shuffle=True)
duration = datetime.now() - start
print("Training completed in time: ", duration)


In [ ]:
test_accuracy=model.evaluate(X_test,y_test,verbose=0)
test_accuracy

# CNN


In [ ]:
X_train=X_train.reshape(X_train.shape[0],X_train.shape[1],X_train.shape[2],1)
X_test=X_test.reshape(X_test.shape[0],X_test.shape[1],X_test.shape[2],1)

In [ ]:
input_shape=(X_train.shape[1],X_train.shape[2],1)
CNNmodel = models.Sequential()
CNNmodel.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape,kernel_initializer='glorot_uniform'))
CNNmodel.add(layers.MaxPooling2D((2, 2)))
CNNmodel.add(layers.Dropout(0.2))
CNNmodel.add(layers.Conv2D(64, (3, 3), activation='relu'))
CNNmodel.add(layers.MaxPooling2D((2, 2)))
CNNmodel.add(layers.Dropout(0.2))
CNNmodel.add(layers.Conv2D(64, (3, 3), activation='relu'))
CNNmodel.add(layers.Flatten())
CNNmodel.add(layers.Dense(64, activation='relu'))
CNNmodel.add(layers.Dropout(0.2))
CNNmodel.add(layers.Dense(32, activation='relu'))
CNNmodel.add(layers.Dense(num_labels, activation='softmax'))

In [ ]:
input_shape=(X_train.shape[1],X_train.shape[2],1)
CNNmodel = models.Sequential()
CNNmodel.add(layers.Conv2D(32,(3,3), activation='relu', input_shape=input_shape))
CNNmodel.add(layers.MaxPooling2D((2, 2)))
CNNmodel.add(layers.Flatten())
CNNmodel.add(layers.Dense(32, activation='relu'))
CNNmodel.add(layers.Dense(num_labels, activation='softmax'))
CNNmodel.summary()

Model: "sequential_43"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_72 (Conv2D)          (None, 126, 1298, 32)     320       
                                                                 
 max_pooling2d_34 (MaxPoolin  (None, 63, 649, 32)      0         
 g2D)                                                            
                                                                 
 flatten_15 (Flatten)        (None, 1308384)           0         
                                                                 
 dense_29 (Dense)            (None, 32)                41868320  
                                                                 
 dense_30 (Dense)            (None, 24)                792       
                                                                 
Total params: 41,869,432
Trainable params: 41,869,432
Non-trainable params: 0
_________________________________________

In [ ]:
CNNmodel.compile(optimizer='adam',loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),metrics=['accuracy'])

In [ ]:
history = CNNmodel.fit(X_train, y_train, epochs=5, validation_split=0.2,batch_size=4)

Epoch 1/5
112/112 [==============================] - 103s 811ms/step - loss: 1.0157 - accuracy: 0.5915 - val_loss: 1.4661 - val_accuracy: 0.3036
Epoch 2/5
112/112 [==============================] - 90s 800ms/step - loss: 0.8011 - accuracy: 0.6987 - val_loss: 2.1415 - val_accuracy: 0.3214
Epoch 3/5
112/112 [==============================] - 94s 842ms/step - loss: 0.5302 - accuracy: 0.7835 - val_loss: 2.2514 - val_accuracy: 0.2946
Epoch 4/5
112/112 [==============================] - 90s 802ms/step - loss: 0.3574 - accuracy: 0.8705 - val_loss: 2.9260 - val_accuracy: 0.2679
Epoch 5/5
112/112 [==============================] - 90s 807ms/step - loss: 0.2037 - accuracy: 0.9286 - val_loss: 3.3676 - val_accuracy: 0.2857


In [ ]:
test_accuracy=CNNmodel.evaluate(X_test,y_test,verbose=0)
test_accuracy

[3.3803212642669678, 0.3214285671710968]

# CNN2

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dense, Flatten

In [ ]:
X_train=X_train.reshape(X_train.shape[0],X_train.shape[1],X_train.shape[2],1)
X_test=X_test.reshape(X_test.shape[0],X_test.shape[1],X_test.shape[2],1)

In [ ]:
model = Sequential()
model.add(Conv2D(16, (3,3), activation='relu', input_shape=(X_train.shape[1],X_train.shape[2],1)),kernel_initializer='glorot_uniform'))
model.add(Conv2D(16, (3,3), activation='relu'))
model.add(Flatten())
# model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))


In [ ]:
# Compiling and fitting the model
#metrics=[tfa.metrics.CohenKappa(num_classes=num_labels, sparse_labels=True, weightage="quadratic"),"accuracy"])

model.compile('Adam', loss='BinaryCrossentropy', metrics=[tf.keras.metrics.Recall(),tf.keras.metrics.Precision(),tfa.metrics.CohenKappa(num_classes=num_labels, sparse_labels=True, weightage="quadratic"),"accuracy"])
model.fit(X_train, y_train, epochs=4, validation_split=0.1, batch_size=32)

Epoch 1/4
126/126 [==============================] - 61s 457ms/step - loss: -2159109.7500 - recall_2: 1.0000 - precision_2: 0.7857 - cohen_kappa: 0.0000e+00 - accuracy: 0.2917 - val_loss: -9994530.0000 - val_recall_2: 1.0000 - val_precision_2: 0.8036 - val_cohen_kappa: 0.0000e+00 - val_accuracy: 0.2679
Epoch 2/4
126/126 [==============================] - 55s 436ms/step - loss: -28001346.0000 - recall_2: 1.0000 - precision_2: 0.7857 - cohen_kappa: 0.0000e+00 - accuracy: 0.2917 - val_loss: -82629016.0000 - val_recall_2: 1.0000 - val_precision_2: 0.8036 - val_cohen_kappa: 0.0000e+00 - val_accuracy: 0.2679
Epoch 3/4
126/126 [==============================] - 55s 435ms/step - loss: -124523760.0000 - recall_2: 1.0000 - precision_2: 0.7857 - cohen_kappa: 0.0000e+00 - accuracy: 0.2917 - val_loss: -290700640.0000 - val_recall_2: 1.0000 - val_precision_2: 0.8036 - val_cohen_kappa: 0.0000e+00 - val_accuracy: 0.2679
Epoch 4/4
126/126 [==============================] - 55s 435ms/step - loss: -33933

In [ ]:
print(model.evaluate(X_test,y_test))

5/5 [==============================] - 5s 726ms/step - loss: -454215168.0000 - recall_2: 1.0000 - precision_2: 0.6786 - cohen_kappa: 0.0000e+00 - accuracy: 0.2429
[-454215168.0, 1.0, 0.6785714030265808, 0.0, 0.24285714328289032]


# CNN3

In [ ]:
X_train.shape

(560, 40, 2600, 1)

In [ ]:
model = keras.models.Sequential([    
    keras.layers.Conv2D(filters=16, kernel_size=(3,3), activation='relu', input_shape=(40,2600,1),kernel_initializer='glorot_uniform'),
    keras.layers.Conv2D(filters=32, kernel_size=(3,3), activation='relu'),
    keras.layers.MaxPool2D(pool_size=(2, 2)),
    keras.layers.BatchNormalization(axis=-1),
    
    keras.layers.Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
    keras.layers.Conv2D(filters=128, kernel_size=(3,3), activation='relu'),
    keras.layers.MaxPool2D(pool_size=(2, 2)),
    keras.layers.BatchNormalization(axis=-1),
    
    keras.layers.Flatten(),
    keras.layers.Dense(512, activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(rate=0.5),
    
    keras.layers.Dense(4, activation='softmax')
])

# model.summary()

In [ ]:
lr = 0.001
epochs = 10

opt = tf.keras.optimizers.legacy.Adam(lr=lr, decay=lr / (epochs * 0.5))
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

In [ ]:
y_train = np.array(y_train)
mlb = LabelBinarizer()
y_train = mlb.fit_transform(y_train)

In [ ]:
X_train,X_val,y_train,y_val=train_test_split(X_train,y_train,test_size=0.2,random_state=0)

In [ ]:
aug = ImageDataGenerator(
    rotation_range=10,
    zoom_range=0.15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.15,
    horizontal_flip=False,
    vertical_flip=False,
    fill_mode="nearest")

history = model.fit(aug.flow(X_train, y_train, batch_size=16), epochs=epochs, validation_data=(X_val, y_val))

Epoch 1/10


# CNN4


In [ ]:
all_mfccs=all_mfccs.reshape(all_mfccs.shape[0],all_mfccs.shape[1],all_mfccs.shape[2],1)

In [ ]:
all_mfccs=np.array(all_mfccs)
labels=np.array(labels)

In [ ]:
len(labels)

700

In [ ]:
model.fit(all_mfccs,labels,epochs=20, batch_size=4, shuffle=True)

Epoch 1/20
175/175 [==============================] - 8s 48ms/step - loss: 1.3705 - accuracy: 0.3071
Epoch 2/20
175/175 [==============================] - 6s 33ms/step - loss: 1.3756 - accuracy: 0.3086
Epoch 3/20
175/175 [==============================] - 6s 33ms/step - loss: 1.3657 - accuracy: 0.3086
Epoch 4/20
175/175 [==============================] - 6s 33ms/step - loss: 1.3647 - accuracy: 0.3100
Epoch 5/20
175/175 [==============================] - 6s 33ms/step - loss: 1.3642 - accuracy: 0.3100
Epoch 6/20
175/175 [==============================] - 8s 43ms/step - loss: 1.3660 - accuracy: 0.3071
Epoch 7/20
175/175 [==============================] - 7s 38ms/step - loss: 1.3662 - accuracy: 0.3057
Epoch 8/20
175/175 [==============================] - 10s 57ms/step - loss: 1.3649 - accuracy: 0.3086
Epoch 9/20
175/175 [==============================] - 10s 57ms/step - loss: 1.3656 - accuracy: 0.3071
Epoch 10/20
175/175 [==============================] - 7s 40ms/step - loss: 1.3646 - accu

# Before Machine Learning

In [ ]:
### Split the dataset into independent and dependent dataset
X=np.array(extracted_features_df['feature'].tolist())
y=np.array(extracted_features_df['class'].tolist())
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)

# Lazy predict

In [ ]:
clf=LazyClassifier(verbose=0,ignore_warnings=True,custom_metric=None)
models,predictions=clf.fit(X_train,X_test,y_train,y_test)
print(models)

100%|██████████| 29/29 [00:24<00:00,  1.19it/s]

                               Accuracy  Balanced Accuracy ROC AUC  F1 Score  \
Model                                                                          
RidgeClassifierCV                  0.39               0.36    None      0.37   
AdaBoostClassifier                 0.38               0.34    None      0.37   
LogisticRegression                 0.35               0.34    None      0.35   
RandomForestClassifier             0.40               0.34    None      0.37   
SGDClassifier                      0.36               0.34    None      0.35   
NearestCentroid                    0.31               0.33    None      0.27   
LinearSVC                          0.35               0.33    None      0.34   
RidgeClassifier                    0.35               0.32    None      0.33   
Perceptron                         0.34               0.31    None      0.34   
BernoulliNB                        0.29               0.31    None      0.27   
XGBClassifier                      0.36 

# SVC

In [ ]:
#clf = make_pipeline(StandardScaler(), SVC())
clf = svm.SVC(kernel='linear', C=3)
clf.fit(X_train, y_train)
pred=clf.predict(X_test)
print("Accuracy:", clf.score(X_test, y_test))
print("QWK:", cohen_kappa_score(y_test, pred, weights='quadratic'))

print("F1 Score")
for i in range(num_labels):
  print("Label",i," F1 Score:", f1_score(y_test, pred,average=None)[i])


Accuracy: 0.34765625
QWK: 0.212911555842479
F1 Score
Label 0  F1 Score: 0.43750000000000006
Label 1  F1 Score: 0.24806201550387597
Label 2  F1 Score: 0.42328042328042326
Label 3  F1 Score: 0.2923076923076923


# LogisticRegression

In [ ]:
clf = LogisticRegression(random_state=0).fit(X_train, y_train)
pred=clf.predict(X_test)
print("Accuracy:", clf.score(X_test, y_test))
print("QWK:", cohen_kappa_score(y_test, pred, weights='quadratic'))

print("F1 Score")
for i in range(num_labels):
  print("Label",i," F1 Score:", f1_score(y_test, pred,average=None)[i])


Accuracy: 0.3984375
QWK: 0.22804760373110322
F1 Score
Label 0  F1 Score: 0.30434782608695654
Label 1  F1 Score: 0.22680412371134023
Label 2  F1 Score: 0.5271317829457364
Label 3  F1 Score: 0.2882882882882883


# KNeighborsClassifier

In [ ]:
clf = KNeighborsClassifier(n_neighbors=9)
clf.fit(X_train, y_train)
pred=clf.predict(X_test)
print("Accuracy:", clf.score(X_test, y_test))
print("QWK:", cohen_kappa_score(y_test, pred, weights='quadratic'))

print("F1 Score")
for i in range(num_labels):
  print("Label",i," F1 Score:", f1_score(y_test, pred,average=None)[i])

Accuracy: 0.33984375
QWK: 0.06499429874572404
F1 Score
Label 0  F1 Score: 0.043478260869565216
Label 1  F1 Score: 0.3063063063063063
Label 2  F1 Score: 0.4444444444444445
Label 3  F1 Score: 0.302158273381295


# ExtraTreesClassifier

In [ ]:
clf = ExtraTreesClassifier(n_estimators=90, random_state=0)
clf.fit(X_train, y_train)
pred=clf.predict(X_test)
print("Accuracy:", clf.score(X_test, y_test))
print("QWK:", cohen_kappa_score(y_test, pred, weights='quadratic'))

print("F1 Score")
for i in range(num_labels):
  print("Label",i," F1 Score:", f1_score(y_test, pred,average=None)[i])


Accuracy: 0.37890625
QWK: 0.14956736711990115
F1 Score
Label 0  F1 Score: 0.1702127659574468
Label 1  F1 Score: 0.3103448275862069
Label 2  F1 Score: 0.502127659574468
Label 3  F1 Score: 0.2807017543859649


# RandomForestClassifier

In [ ]:
clf = RandomForestClassifier(n_estimators=10,max_depth=9,random_state=0)
clf.fit(X_train, y_train)
pred=clf.predict(X_test)
print("Accuracy:", clf.score(X_test, y_test))
print("QWK:", cohen_kappa_score(y_test, pred, weights='quadratic'))

print("F1 Score")
for i in range(num_labels):
  print("Label",i," F1 Score:", f1_score(y_test, pred,average=None)[i])

Accuracy: 0.3203125
QWK: 0.10697674418604652
F1 Score
Label 0  F1 Score: 0.16666666666666666
Label 1  F1 Score: 0.18803418803418803
Label 2  F1 Score: 0.4661016949152542
Label 3  F1 Score: 0.21621621621621623


# LGBMClassifier

In [ ]:
clf = lgb.LGBMClassifier()
clf.fit(X_train, y_train)
pred=clf.predict(X_test)
print("Accuracy:", clf.score(X_test, y_test))
print("QWK:", cohen_kappa_score(y_test, pred, weights='quadratic'))

print("F1 Score")
for i in range(num_labels):
  print("Label",i," F1 Score:", f1_score(y_test, pred,average=None)[i])


Accuracy: 0.3515625
QWK: 0.24094307073030474
F1 Score
Label 0  F1 Score: 0.20408163265306123
Label 1  F1 Score: 0.26229508196721313
Label 2  F1 Score: 0.43192488262910794
Label 3  F1 Score: 0.359375


# XGBClassifier

In [ ]:
xgbc = XGBClassifier(learning_rate =0.005,
 n_estimators=10,
 max_depth=5,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'multi:softprob',
 nthread=4,
 seed=27)

#xgbc = XGBClassifier()
xgbc.fit(X_train, y_train)
pred=clf.predict(X_test)
print("Accuracy:", clf.score(X_test, y_test))
print("QWK:", cohen_kappa_score(y_test, pred, weights='quadratic'))

print("F1 Score")
for i in range(num_labels):
  print("Label",i," F1 Score:", f1_score(y_test, pred,average=None)[i])


Accuracy: 0.3515625
QWK: 0.24094307073030474
F1 Score
Label 0  F1 Score: 0.20408163265306123
Label 1  F1 Score: 0.26229508196721313
Label 2  F1 Score: 0.43192488262910794
Label 3  F1 Score: 0.359375


# BaggingClassifier

In [ ]:
clf = BaggingClassifier(n_estimators=150,random_state=0)
clf.fit(X_train, y_train)
pred=clf.predict(X_test)
print("Accuracy:", clf.score(X_test, y_test))
print("QWK:", cohen_kappa_score(y_test, pred, weights='quadratic'))

print("F1 Score")
for i in range(num_labels):
  print("Label",i," F1 Score:", f1_score(y_test, pred,average=None)[i])

Accuracy: 0.37890625
QWK: 0.17954780552682137
F1 Score
Label 0  F1 Score: 0.20408163265306123
Label 1  F1 Score: 0.25
Label 2  F1 Score: 0.4954954954954955
Label 3  F1 Score: 0.36363636363636365


# NuSVC

In [ ]:
clf = make_pipeline(StandardScaler(), NuSVC())
clf.fit(X_train, y_train)
pred=clf.predict(X_test)
print("Accuracy:", clf.score(X_test, y_test))
print("QWK:", cohen_kappa_score(y_test, pred, weights='quadratic'))

print("F1 Score")
for i in range(num_labels):
  print("Label",i," F1 Score:", f1_score(y_test, pred,average=None)[i])


Accuracy: 0.32421875
QWK: 0.25072523829258186
F1 Score
Label 0  F1 Score: 0.2
Label 1  F1 Score: 0.2608695652173913
Label 2  F1 Score: 0.37696335078534027
Label 3  F1 Score: 0.3609022556390977


# LinearSVC

In [ ]:
clf = make_pipeline(StandardScaler(),LinearSVC(random_state=0, tol=1e-5))
clf.fit(X_train, y_train)
pred=clf.predict(X_test)
print("Accuracy:", clf.score(X_test, y_test))
print("QWK:", cohen_kappa_score(y_test, pred, weights='quadratic'))

print("F1 Score")
for i in range(num_labels):
  print("Label",i," F1 Score:", f1_score(y_test, pred,average=None)[i])


Accuracy: 0.34765625
QWK: 0.19166336894868752
F1 Score
Label 0  F1 Score: 0.43333333333333335
Label 1  F1 Score: 0.1754385964912281
Label 2  F1 Score: 0.4334975369458128
Label 3  F1 Score: 0.3259259259259259


# SGDClassifier

In [ ]:
clf = make_pipeline(StandardScaler(),SGDClassifier(max_iter=1000, tol=1e-3))
clf.fit(X_train, y_train)
pred=clf.predict(X_test)
print("Accuracy:", clf.score(X_test, y_test))
print("QWK:", cohen_kappa_score(y_test, pred, weights='quadratic'))

print("F1 Score")
for i in range(num_labels):
  print("Label",i," F1 Score:", f1_score(y_test, pred,average=None)[i])


Accuracy: 0.3046875
QWK: 0.20774251631780327
F1 Score
Label 0  F1 Score: 0.30952380952380953
Label 1  F1 Score: 0.22429906542056074
Label 2  F1 Score: 0.3636363636363636
Label 3  F1 Score: 0.28965517241379307


# LinearDiscriminantAnalysis

In [ ]:
clf = LinearDiscriminantAnalysis()
clf.fit(X_train, y_train)
pred=clf.predict(X_test)
print("Accuracy:", clf.score(X_test, y_test))
print("QWK:", cohen_kappa_score(y_test, pred, weights='quadratic'))

print("F1 Score")
for i in range(num_labels):
  print("Label",i," F1 Score:", f1_score(y_test, pred,average=None)[i])

Accuracy: 0.32421875
QWK: 0.09442060085836912
F1 Score
Label 0  F1 Score: 0.3448275862068965
Label 1  F1 Score: 0.19999999999999998
Label 2  F1 Score: 0.42574257425742573
Label 3  F1 Score: 0.27272727272727276
